<a href="https://colab.research.google.com/github/kareemullah1234/AI_Agent_content/blob/main/Agent_6_streaming_persist_sql_remebers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U \
  langchain-core \
  langchain \
  langchain-groq \
  langchain-tavily \
  langgraph \
  langgraph-checkpoint \
  aiosqlite \
  sqlite-utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.2 MB/s eta 0:00:00


In [3]:
!pip install langgraph-checkpoint-sqlite  #important


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 10.7 MB/s eta 0:00:00


In [1]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

conn = sqlite3.connect("memory.db", check_same_thread=False)
memory = SqliteSaver(conn)
print("✅ SqliteSaver imported successfully")


✅ SqliteSaver imported successfully


In [2]:
#!/usr/bin/env python
# coding: utf-8

# # Lesson 4: Persistence and Streaming (Modified for Groq)

# In[1]:


# Install required packages (Run this cell first, then RESTART the runtime)
# !pip install -q -U langchain-core langchain langchain-groq langchain-tavily langgraph langgraph-checkpoint aiosqlite sqlite-utils





# 🔑 Hardcode API keys
import os



# In[3]:


# Import necessary modules
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

# 🔁 Use ChatGroq instead of ChatOpenAI
from langchain_groq import ChatGroq

# 🔍 Use the updated Tavily tool
from langchain_tavily import TavilySearch


# In[4]:


# Initialize the Tavily tool
tool = TavilySearch(max_results=2)


# In[5]:


# Define the agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]



# Define the Agent class
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm) # Renamed for clarity
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # 🔁 Updated to call Groq model
    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


# In[8]:


# 🚀 Initialize the Groq model
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# Use a Groq model that supports tool calling
model = ChatGroq(model="llama3-8b-8192", temperature=0.3, max_tokens=512)


# In[9]:


# Create the agent instance with Groq model and SQLite persistence
abot = Agent(model, [tool], system=prompt, checkpointer=memory)


In [3]:


# In[10]:


# 🧪 Test the agent
messages = [HumanMessage(content="What is the weather in sf?")]
thread = {"configurable": {"thread_id": "1"}}

# Stream the response
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7e0j5ytp8', 'function': {'arguments': '{"query":"weather in sf","search_depth":"basic","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 2807, 'total_tokens': 2903, 'completion_time': 0.080663435, 'prompt_time': 0.316417031, 'queue_time': 0.003841487, 'total_time': 0.397080466}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_343314801a', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3bbe1b24-2141-4fa2-a160-01c9ffd68686-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'weather in sf', 'search_depth': 'basic', 'topic': 'general'}, 'id': '7e0j5ytp8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2807, 'output_tokens': 96, 'total_tokens': 2903})]
Calling: {'name': 'tavily_search', 'args': {'query': 'weather in sf', 'search_depth': 'basic', 'topic': 'general'}

In [4]:

# In[11]:


# Ask a follow-up question (persistence should remember the conversation)
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}} # Same thread ID

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'avb7kn7sn', 'function': {'arguments': '{"query":"weather in la","search_depth":"basic","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 3722, 'total_tokens': 3768, 'completion_time': 0.038490924, 'prompt_time': 0.413777928, 'queue_time': 0.004293296, 'total_time': 0.452268852}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bd839012-54b3-47a0-91e6-4bff381565a9-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'weather in la', 'search_depth': 'basic', 'topic': 'general'}, 'id': 'avb7kn7sn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 3722, 'output_tokens': 46, 'total_tokens': 3768})]
Calling: {'name': 'tavily_search', 'args': {'query': 'weather in la', 'search_depth': 'basic', 'topic': 'general'}

In [5]:
# In[12]:


# Ask another question comparing previous results
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}} # Same thread ID

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='According to the results, San Francisco (66°F/18.9°C) is slightly cooler than Los Angeles (68.5°F/20.3°C).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 4574, 'total_tokens': 4608, 'completion_time': 0.028119841, 'prompt_time': 0.503627547, 'queue_time': 0.005475129, 'total_time': 0.531747388}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0da2c81d-8737-4fbf-99a7-cb68dfd149c9-0', usage_metadata={'input_tokens': 4574, 'output_tokens': 34, 'total_tokens': 4608})]


In [6]:
# In[13]:


# Start a new conversation thread
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}} # New thread ID

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content="I'm happy to help! However, I need more information to provide a accurate answer. Can you please specify what you are referring to? Are you asking about the temperature of a specific location, a person, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 2805, 'total_tokens': 2852, 'completion_time': 0.039573516, 'prompt_time': 0.31453707, 'queue_time': 0.003754728, 'total_time': 0.354110586}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_343314801a', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--65e0d591-3efc-4900-a5f3-0120058e0d05-0', usage_metadata={'input_tokens': 2805, 'output_tokens': 47, 'total_tokens': 2852})]


In [ ]:











# ## Streaming tokens (Example - requires async environment)

# In[ ]:


# # For token streaming, you would typically use async
# from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver # Ensure aiosqlite is installed
# import asyncio

# # Set up async memory
# # async_memory = AsyncSqliteSaver.from_conn_string("memory.db") # Or ":memory:"
# # abot_stream = Agent(model, [tool], system=prompt, checkpointer=async_memory)

# # Example async streaming function (requires running in an async context)
# # async def stream_response():
# #     messages = [HumanMessage(content="What is the weather in SF?")]
# #     thread = {"configurable": {"thread_id": "4"}}
# #     async for event in abot_stream.graph.astream_events({"messages": messages}, thread, version="v1"):
# #         kind = event["event"]
# #         if kind == "on_chat_model_stream":
# #             content = event["data"]["chunk"].content
# #             if content:
# #                 print(content, end="|")
# # await stream_response() # This line would be run in an async environment
